In [ ]:
import os
import pickle as pkl
from joblib import load, dump
import cv2
import matplotlib
import tensorflow as tf
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
from tensorflow.python.client import device_lib
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

In [ ]:

MODEL_LIST = []
MODEL_NAMES = ['skin_disease_resnet_No_AUG_88acc', 'resnet_model1_cifar10']

for i in range(len(MODEL_NAMES)):
    ### Loading DNN models 
    with open('Models/'+ MODEL_NAMES[i]+'.json', 'r') as j_file:
        loaded_model_json = j_file.read()
    model = tf.keras.models.model_from_json(loaded_model_json)
    model.load_weights("Models/"+MODEL_NAMES[i]+".h5")
    if (MODEL_NAMES[i] == 'skin_disease_resnet_No_AUG_88acc'):
        model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy',
                      metrics=['acc'])
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
                      loss = "categorical_crossentropy",  metrics= ['accuracy'])
    MODEL_LIST.append(model)



# Cifar 10

In [ ]:
### Cifar model benchmark
def loadPreprocessCifar10():
    ### Load In Data
    (trainX, trainY), (testX, testY) = tf.keras.datasets.cifar10.load_data()
    trainY = tf.keras.utils.to_categorical(trainY)
    testY = tf.keras.utils.to_categorical(testY)
    return [trainX/255, trainY, testX/255, testY]

trainX, trainY, testX, testY = loadPreprocessCifar10()
extractor0  = tf.keras.Model(inputs = MODEL_LIST[0].input,
                         outputs = MODEL_LIST[0].get_layer(MODEL_LIST[0].layers[-5].name).output
                        )
extractor1  = tf.keras.Model(inputs = MODEL_LIST[1].input,
                         outputs = MODEL_LIST[1].get_layer(MODEL_LIST[1].layers[-5].name).output
                        )
### Predictions
#p = model1.predict(trainX,verbose = 1)
print("Model Evaluation")
MODEL_LIST[1].evaluate(testX, testY, verbose =1)

In [ ]:
print("Extracting Features......")
print("May take a while")
feat0 = []
feat1 = []
for i in tqdm(range(len(trainX))):
    tmp = np.expand_dims(cv2.resize(trainX[i], (200,200)), axis = 0)
    feat0.append(extractor0.predict(tmp))
    feat1.append(extractor1.predict(np.expand_dims(trainX[i], axis = 0)))    

In [ ]:
#np.save("data/train_feat0.npy", np.vstack(feat0))
#np.save("data/train_feat1.npy", np.vstack(feat1))

In [ ]:
print("Extracting Features......")
print("May take a while")
test_feat0 = []
test_feat1 = []
for i in tqdm(range(len(testX))):
    tmp = np.expand_dims(cv2.resize(testX[i], (200,200)), axis = 0)
    test_feat0.append(extractor0.predict(tmp))
    test_feat1.append(extractor1.predict(np.expand_dims(testX[i], axis = 0)))    

In [ ]:
np.save("data/test_feat0.npy", np.vstack(test_feat0))
np.save("data/test_feat1.npy", np.vstack(test_feat1))

In [ ]:
extractor1.summary()

In [ ]:
from sklearn.svm import SVC
svc1 = SVC()
svc1.fit(np.vstack(feat1),np.argmax(trainY, axis = 1) )

In [ ]:
from sklearn.metrics import accuracy_score 
pred = svc1.predict(np.vstack(feat1))
acc = accuracy_score(pred, np.argmax(trainY, axis = 1) )
print("The accuracy of the support vector Machine: ", acc)

In [ ]:
acc

In [ ]:
print("Performing predictions....")
print("May take a while")
pred0 = []
pred1 = []
for i in tqdm(range(len(testX))):
    tmp = np.expand_dims(cv2.resize(testX[i], (200,200)), axis = 0)
    pred0.append(MODEL_LIST[0].predict(tmp))
    pred1.append(MODEL_LIST[1].predict(np.expand_dims(testX[i], axis = 0)))    

In [ ]:
np.save("data/test_feat0.npy", np.vstack(feat0))
np.save("data/test_feat1.npy", np.vstack())

In [ ]:
n = 0
correct_model_predictions_idx = []
for i in range(len(pred0)):
    if(np.amax(pred0[i])>np.amax(pred1[i])):
        n = n+1
    else:
        correct_model_predictions_idx.append(i)
print("{} samples were passed to the wrong model.".format(n))
print("This is because the wrong model had a higher confidence value.")
pidx = np.argmax(np.vstack(pred1), axis = 1)[correct_model_predictions_idx]
groundTruth = np.argmax(testY[correct_model_predictions_idx],axis = 1)
from sklearn.metrics import accuracy_score
acc = accuracy_score(groundTruth,pidx)
print("Accuracy",acc)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_curve
f1 = f1_score(groundTruth, pidx, average = "macro")
acc = accuracy_score(groundTruth,pidx)
recall = recall_score(groundTruth, pidx, average = "macro")
precision =precision_score(groundTruth, pidx, average = "macro")
print("Accuracy: {}".format(acc))
print("F1 Score: {}".format(f1))
print("recall Score: {}".format(recall))
print("Precision Score: {}".format(precision))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
conf = tf.math.confusion_matrix(groundTruth,pidx)
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 10}
matplotlib.rc('font', **font)
labels_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship',
                'truck']
df_cm = pd.DataFrame(np.array(conf), index = [i for i in labels_names],
                  columns = [i for i in labels_names])
plt.figure(figsize = (20,10))
sn.heatmap(df_cm, annot=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
y_test = np.copy(testY[correct_model_predictions_idx])
y_score = np.copy(np.vstack(pred1)[correct_model_predictions_idx])
n_classes =  8
lw =2
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])



# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize = (20, 30))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

# FADL on The Skin Disease Dataset

In [ ]:
gnd_truth = pd.read_csv("data\SkinDiseaseDataset\ISIC_2019_Training_GroundTruth.csv")
#aug_df = pd.read_csv("data/augmentImageNameLabels.csv")
gnd_truth.head()

IMG_NAMES = gnd_truth['image']
TRAIN_IMG_RT_PATH = 'data/ISIC_2019_Training_Input/'
TEST_IMG_RT_PATH = 'data/ISIC_2019_Test_Input'
### No Examples of UNK so drop it
labels = np.array(gnd_truth.drop(columns = ["image", "UNK"]))


In [ ]:
c, freq = np.unique(np.argmax(labels, axis= 1), return_counts = True)
plt.xlabel("Class")
plt.ylabel("# of Occurences")
plt.bar(c,freq)
print("# of occurences")
print(freq)
print("Classes")
print(c)

In [ ]:
def returnClassIndex(trainY,class_index):
    labels = np.argmax(trainY, axis = 1)
    return [ i for i in range(len(labels)) if labels[i] == class_index]


### Randomly Balance Sample from classes  
smpl_test_idxs = []
smpl_train_idxs = []
for i in range(len(labels[1])):
    idxs = np.array(returnClassIndex(labels, i))
    sel = np.random.RandomState(seed = 42).permutation(len(idxs))
    #print(len(idxs))
    ### Shuffle
    train_sel = list(sel[:int(0.75*len(idxs))])
    test_sel = list(sel[int(0.75*len(idxs)):])
    smpl_train_idxs += list(idxs[train_sel])
    smpl_test_idxs += list(idxs[test_sel])

paths = np.array([TRAIN_IMG_RT_PATH + m +'.jpg' for m in IMG_NAMES ])

In [ ]:
im_size =200
channels = 3
num_classes = 8
lr = 0.001 ### learning rate
input_shape = ( im_size, im_size, channels)
l1_lambda = 0 ### L1 regularization lambda parameter
l2_lambda = 0
keep_prob =  0
batch_size = 16

In [ ]:
### Dataframes for Generator
train_df = pd.DataFrame({"filename":paths[smpl_train_idxs],
                        "class": np.argmax(labels, axis = 1)[smpl_train_idxs].astype(str)})
test_df = pd.DataFrame({"filename":paths[smpl_test_idxs],
                        "class": np.argmax(labels, axis = 1)[smpl_test_idxs].astype(str)})

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0/255,
        samplewise_center = True,
        samplewise_std_normalization = True, 
        horizontal_flip=True,
        vertical_flip = True,
        shear_range=0.5,
        zoom_range=0.5,
        rotation_range = 30
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    class_mode = 'categorical',
    y_col="class",
    batch_size = batch_size, 
    target_size = (im_size, im_size)
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    class_mode = 'categorical',
    y_col="class",
    batch_size = batch_size,
    target_size = (im_size, im_size)
)

In [ ]:
MODEL_LIST[0].evaluate(test_generator)

In [ ]:
print("Performing predictions....")
print("May take a while")
pred1 = []
pred0 = []
labels = []
for i in tqdm(range(len(test_generator) -1)):
    tmp = next(test_generator)
    for i0 in range(16):
        pred1.append(MODEL_LIST[1].predict(
            np.expand_dims(cv2.resize(tmp[0][i0], (32, 32)), axis = 0)))
    pred0.append(MODEL_LIST[0].predict(tmp[0]))
    labels.append(tmp[1])
p0 = np.vstack(pred0)
p1 = np.vstack(pred1)
test_labels = np.vstack(labels)

In [ ]:
p0 = np.vstack(pred0)
p1 = np.vstack(pred1)
test_labels = np.vstack(labels)

In [ ]:
n = 0
correct_model_predictions_idx = []
for i in tqdm(range(len(p0))):
    if(np.amax(p1[i])>np.amax(p0[i])):
        n = n+1
    else:
        correct_model_predictions_idx.append(i)
print("{} samples were passed to the wrong model.".format(n))
print("This is because the wrong model had a higher confidence value.")
pidx = np.argmax(p0, axis = 1)[correct_model_predictions_idx]
groundTruth = np.argmax(test_labels, axis = 1)[correct_model_predictions_idx]
from sklearn.metrics import accuracy_score
acc = accuracy_score(groundTruth,pidx)
print("Accuracy",acc)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
conf = tf.math.confusion_matrix(groundTruth,pidx)
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 10}
matplotlib.rc('font', **font)
labels_names = list(gnd_truth.columns[1:-1])
df_cm = pd.DataFrame(np.array(conf), index = [i for i in labels_names],
                  columns = [i for i in labels_names])
plt.figure(figsize = (20,10))
sn.heatmap(df_cm, annot=True)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_curve
f1 = f1_score(groundTruth, pidx, average = "macro")
acc = accuracy_score(groundTruth,pidx)
recall = recall_score(groundTruth, pidx, average = "macro")
precision =precision_score(groundTruth, pidx, average = "macro")
print("Metrics")
print()
print("Accuracy: {}".format(acc))
print("F1 Score: {}".format(f1))
print("recall Score: {}".format(recall))
print("Precision Score: {}".format(precision))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
y_test = np.copy(test_labels[correct_model_predictions_idx])
y_score = np.copy(p0[correct_model_predictions_idx])
n_classes =  8
lw =2
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])



# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize = (20, 30))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()